In [1]:
from datetime import datetime

import stackstac
import pystac_client
import planetary_computer as pc

import ipyleaflet
import IPython.display as dsp
import ipywidgets as widgets 

In [5]:
# d1 = widgets.DatePicker(
#     value=datetime.fromisoformat("2015-01-01"),
#     description='From',
#     width="100px"
# )
# d2 = widgets.DatePicker(
#     value=datetime.fromisoformat("2016-01-01"),
#     description='To',
#     width="100px"
# )

m = ipyleaflet.Map(scroll_wheel_zoom=True)
m.center = -4.313, 20.370
m.zoom = 12
# m.layout.height = "500px"
# m.layout.width = "500px"

grid = widgets.GridspecLayout(10, 2)
grid[:, 0] = m
# grid[0, 1] = d1
# grid[1, 1] = d2
grid[:, 0].layout.height = 'auto'
grid

GridspecLayout(children=(Map(center=[-4.313, 20.37], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [14]:
bbox = (m.west, m.south, m.east, m.north)
bbox
start = "2019-05-01"
end = "2019-07-01"
bbox, start, end

((20.261192321777347,
  -4.312861663258931,
  20.478515625000004,
  -4.312861663258931),
 '2019-05-01',
 '2019-07-01')

In [15]:
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=['sentinel-2-l2a'], 
    bbox=bbox, 
    datetime=f"{start}/{end}"
)
print(f"{search.matched()} Items")


6 Items


In [18]:
items = search.items_as_collection()
items = pystac_client.ItemCollection([pc.sign_assets(item) for item in items.features])

items.to_dict()


{'type': 'FeatureCollection',
 'links': [],
 'features': [{'type': 'Feature',
   'stac_version': '1.0.0-beta.2',
   'id': 'S2B_MSIL2A_20190604T084609_N0212_R107_T34MDA_20201005T220616.SAFE',
   'properties': {'platform': 'Sentinel-2B',
    'constellation': 'Sentinel 2',
    'instruments': ['msi'],
    'eo:cloud_cover': 58.653505,
    'sat:orbit_state': 'descending',
    'sat:relative_orbit': 107,
    'proj:epsg': 32734,
    's2:product_uri': 'S2B_MSIL2A_20190604T084609_N0212_R107_T34MDA_20201005T220616.SAFE',
    's2:generation_time': '2020-10-05T22:06:16.868Z',
    's2:processing_baseline': '02.12',
    's2:product_type': 'S2MSI2A',
    's2:datatake_id': 'GS2B_20190604T084609_011716_N02.12',
    's2:datatake_type': 'INS-NOBS',
    's2:datastrip_id': 'S2B_OPER_MSI_L2A_DS_ESRI_20201005T220619_S20190604T090234_N02.12',
    's2:granule_id': 'S2B_OPER_MSI_L2A_TL_ESRI_20201005T220619_A011716_T34MDA_N02.12',
    's2:mgrs_tile': '34MDA',
    's2:reflectance_conversion_factor': 0.9728536684374

In [17]:
%%time
stack = stackstac.stack(items, bounds_latlon=bbox)
stack

ValueError: Cannot automatically compute the resolution, since asset 'preview' on item 0 'S2A_MSIL2A_20190517T083601_N0212_R064_T34MDA_20201006T151237.SAFE' doesn't provide enough metadata to determine its native resolution.
We'd need at least one of (in order of preference):
- The `proj:transform` and `proj:epsg` fields set on the asset, or on the item
- The `proj:shape` and one of `proj:bbox` or `bbox` fields set on the asset, or on the item

Please specify the `resolution=` argument to set the output resolution manually. (Remember that resolution must be in the units of your CRS (http://epsg.io/32734)---not necessarily meters.